## Two-area network with a stochastic load

The purpose of this notebook is to show how pan can be used to simulate the two-area power network with a stochastic load. The behavior of the load is described by two Python functions and the time series defining the stochastic load is also created in Python.

A single line schematic of Kundur's two area network is shown below:

<a href="https://www.researchgate.net/figure/Kundurs-two-area-power-system_fig1_281703659"><img src="https://www.researchgate.net/profile/Zahra-Rafi/publication/281703659/figure/fig1/AS:338164796411905@1457636178242/Kundurs-two-area-power-system.png" alt="Kundur's two-area power system"/></a>

In the netlist used here, the loads connected at buses 7 and 9 are stochastic, i.e., their values are described by an Ornstein-Uhlenbeck process.

In [ ]:
import pypan.ui as pan
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
from scipy.signal import periodogram
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'Times New Roman', 'size': 8})
%matplotlib inline

The [netlist file](two-area.pan) contains the following line:

`model STLOAD nport macro=yes setup="stoch_load_setup" evaluate="stoch_load_eval"`

which instructs pan to look for two Python functions called `stoch_load_setup` and `stoch_load_eval` when initializing and evaluating the N-port instance, respectively. The initialization function must return a dictionary with the parameters names as keys and their default value as values, while the evaluation function must return the implicit equations describing the N-port and their derivative w.r.t. currents and voltages. 
    
For a more detailed description of how to implement an N-port device in Python, see [this notebook](../NPort.ipynb).

In [ ]:
def stoch_load_setup():
    return {'P': 1e6}

def stoch_load_eval(n, V, I, time, **kwargs):
    MIN_MAGNITUDE = 200e3 ** 2
    P = kwargs['P'] * V[2] # active power
    Q = 0
    magnitude = V[0]**2 + V[1]**2
    if magnitude < MIN_MAGNITUDE:
        magnitude = MIN_MAGNITUDE
    magnitude_squared = magnitude ** 2
    
    f = np.array([
        I[0] - (V[0] * P + V[1] * Q) / magnitude,
        I[1] - (V[1] * P - V[0] * Q) / magnitude,
        I[2]
    ])

    C = np.zeros((n, n))
    C[0,0] = -( P / magnitude - (V[0] * P + V[1] * Q) * 2 * V[0] / magnitude_squared)
    C[0,1] = -( Q / magnitude - (V[0] * P + V[1] * Q) * 2 * V[1] / magnitude_squared)
    C[0,2] = -(kwargs['P'] * V[0] / magnitude)
    C[1,0] = -(-Q / magnitude - (V[1] * P - V[0] * Q) * 2 * V[0] / magnitude_squared)
    C[1,1] = -( P / magnitude - (V[1] * P - V[0] * Q) * 2 * V[1] / magnitude_squared)
    C[1,2] = -(kwargs['P'] * V[1] / magnitude)

    R = np.eye(n)
        
    return f, C, R

Here we define the two functions that are used to generate the Ornstein-Uhlenbeck process and to build a two-row matrix containing the time instants and the OU samples.

In [ ]:
def OU(dt, alpha, mu, sigma, N, random_state = None):
    coeff = np.array([alpha * mu * dt, 1 / (1 + alpha * dt)])
    if random_state is not None:
        rnd = sigma * np.sqrt(dt) * random_state.normal(size=N)
    else:
        rnd = sigma * np.sqrt(dt) * np.random.normal(size=N)
    ou = np.zeros(N)
    ou[0] = mu
    for i in range(N-1):
        ou[i+1] = (ou[i] + coeff[0] + rnd[i]) * coeff[1]
    return ou

def make_noise_samples(tstart, tend, dt, alpha, mu, sigma, seed):
    t = dt + np.r_[tstart : tend + dt/2 : dt]
    N_samples = t.size
    state = RandomState(MT19937(SeedSequence(seed)))
    noise_samples = np.zeros((2, N_samples))
    noise_samples[0,:] = t
    noise_samples[1,:] = OU(dt, alpha, mu, sigma, N_samples, state)
    return noise_samples

The following two lines in the [netlist file](two-area.pan)

`V7   rnd7   gnd  vsource wave="noise_samples_bus_7"` <br/>
`V9   rnd9   gnd  vsource wave="noise_samples_bus_9"`

tell pan that it should look for two variables (called `noise_samples_bus_7` and `noise_samples_bus_9`) in Python memory to use as waveforms for the `V7` and `V9` voltage source objects.

We therefore build these two variables by calling the `make_noise_sample` function defined above.

In [ ]:
alpha = 0.5
mu = 0
sigma = 0.5
frand = 10
dt = 1 / frand
tstart = 0
tstop = 1 * 60 * 60

noise_samples_bus_7 = make_noise_samples(tstart, 1.01 * tstop, dt,
                                         alpha, mu, sigma, 1234567)
noise_samples_bus_9 = make_noise_samples(tstart, 1.01 * tstop, dt,
                                         alpha, mu, sigma, 6234160)

### Load the netlist

In [ ]:
netlist_file = 'two-area.pan'
ok,libs = pan.load_netlist(netlist_file)
if not ok:
    raise Exception('load_netlist failed')

### Run a transient analysis

By defining `mem_vars`, we ask pan to return the values of these variables saved to memory when `pan.tran` returns. In this way, we don't have to load the same data from a file.

In [ ]:
mem_vars = ['time', 'omega1', 'omega2', 'omega3', 'omega4']
data = pan.tran('Tr', tstop, mem_vars, libs, nettype=1,
                method=1, timepoints=1/frand, forcetps=1)

### Compute the PSD of the rotors' angular velocities

In [ ]:
time = data[0]
omega = {i: 60 * data[i] for i in range(1,len(data))}
P = {}
for i,x in omega.items():
    f,P[i] = periodogram(x, frand, window='boxcar')

### Plot the results

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6, 4))
cmap = plt.get_cmap('Dark2', len(P))
for i in P:
    ax.plot(f, 10 * np.log10(P[i]), color=cmap(i-1), lw=1, label=r'$\omega_{}$'.format(i))
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.legend(loc='upper right')
ax.set_ylim([-185, -50])
ax.grid(which='major', axis='both', linestyle=':', linewidth=0.5, color=[.6,.6,.6])
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Power [dB]')
ax.set_yticks(np.r_[-180 : -30 : 25])
fig.tight_layout()
fig.savefig('stochastic_load.pdf')